In [180]:
import numpy as np
import pandas as pd
import math
import spacy
import re

In [181]:
delegates_og = pd.read_excel("Data/constitutional_convention_1787.xlsx", header = 2)
delegates = delegates_og.copy()

In [182]:
#fix second last name in parentheses
#fitzsimons and fitzSimons are the same because we dont care abt caps

In [183]:
print("dimensions:", delegates.shape)
last_name_2 = delegates[delegates['last name'].apply(lambda x: "(" in x)].index
delegates[delegates['last name'].apply(lambda x: "(" in x)]

dimensions: (55, 4)


,first name,last name,state,sign?
2,Oliver,Ellsworth (Elsworth),Connecticut,no
24,David,Brearly (Brearley),New Jersey,yes
25,William,Paterson (Patterson),New Jersey,yes
40,Thomas,Fitzsimons (FitzSimons; Fitzsimmons),Pennsylvania,yes


In [184]:
#cleaning parentheses
delegates['last name 2'] = pd.Series()
delegates['last name'].loc[40] = "Fitzsimons (Fitzsimmons)"
delegates['last name 2'].loc[last_name_2] = delegates['last name'][last_name_2].apply(lambda x: x[x.find("(")+1:x.find(")")])
delegates['last name 2'].loc[last_name_2]

2        Elsworth
24       Brearley
25      Patterson
40    Fitzsimmons
Name: last name 2, dtype: object

In [185]:
delegates = delegates[['first name', 'last name', 'last name 2', 'state', 'sign?']]
#delegates

In [186]:
loans_og = pd.read_excel("Data/loan_office_certificates_9_states.xlsx")
loans = loans_og.copy()

In [187]:
loans['notes'] = pd.Series()

In [188]:
loans.columns

Index(['State', 'Year', 'Month', 'Day', 'Title 1', 'First Name 1 ',
       'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Face Value',
       'Specie Value ', 'notes'],
      dtype='object')

In [189]:
def betterIsNan(x):
    try: 
        return math.isnan(x)
    except:
        return False

In [190]:
print("checking if there are any columns with valid last name 1 and first name 2")
any([betterIsNan(x) and betterIsNan(y) and not betterIsNan(w) and not betterIsNan(z) for w, x, y, z in zip(loans['First Name 1 '], loans['Last Name 1 '], loans['First Name 2'], loans['Last Name 2'])])

checking if there are any columns with valid last name 1 and first name 2


False

In [191]:
print("checking if there are any columns with valid last name 2 and first name 1")
any([betterIsNan(w) and betterIsNan(z) and not betterIsNan(x) and not betterIsNan(y) for w, x, y, z in zip(loans['First Name 1 '], loans['Last Name 1 '], loans['First Name 2'], loans['Last Name 2'])])

checking if there are any columns with valid last name 2 and first name 1


False

In [192]:
def replaceStrings(x):
    try:
        return x.replace("  ", " ")
    except:
        return x

In [193]:
loans['First Name 1 '] = loans['First Name 1 '].apply(lambda x: replaceStrings(x))

In [194]:
# get rid of entries where there is nothing for first name 1/2, last name 1/2 

In [195]:
no_first_name = loans[loans['First Name 1 '].apply(betterIsNan)]
#no_first_name

In [196]:
remove_indices = dict()
remove_indices['no_first_name'] = no_first_name[no_first_name['Last Name 1 '].apply(betterIsNan)].index.tolist()
#no_first_name.loc[remove_indices['no_first_name']]

In [197]:
#last name column is null
no_last_name = loans[loans['Last Name 1 '].apply(betterIsNan)]
#removing overlaps with the no first name column
no_last_name = no_last_name.loc[[x not in set(no_last_name.index).intersection(no_first_name.index) for x in no_last_name.index]]
#no_last_name

In [198]:
def lenGreaterOne(x):
    try: 
        return len(x.split(" ")) > 1 
    except: 
        return False

In [199]:
#clean entries where last name 1 is missing, first name 1 is only one word

In [200]:
one_word_first = no_last_name[no_last_name['First Name 1 '].apply(lambda x: not lenGreaterOne(x))]
remove_indices['one_word_first'] = one_word_first.index.tolist()
#one_word_first.loc[remove_indices['one_word_first']]

In [201]:
# clean entries where there is no last name and the first name is more than one character - follow steps below

In [202]:
weird_first_names = no_last_name[no_last_name['First Name 1 '].apply(lambda x: lenGreaterOne(x))]

<h2> Handle Corrections of Executors </h2>

How do we want to consider executors? Are they irrelevant (ie: do we only care about the person who they are executing for, or do we also care about them - and just add a note saying "executor")? 

Empirically, what interest do the executors have? They probably don't have any direct financial interest in the debt but there could definitely be incentives for them to be interested in the stability of the debt, such as shared business interests, desire to properly help execute a will etc?

What do we do when we cannot properly identify executors? Or trustees?

In [203]:
#trend below - if its Executors just get Atkinsons
#if its David Mackey and Debt Executor just do David Mackey
#for jacob lentz and phillip fisher lentz I kept jacob fisher and removed philip lentz
#there are records for jacob fisher being a soldier but no records for philip lentz
#otherwise find the word "to" or "of" and get the name right after that

In [204]:
executor = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: "exec" in x.lower() or "executor" in x.lower() or "ex." in x.lower())]
executor.shape

(40, 13)

In [205]:
executor['First Name 1 '].unique().tolist()

['Sam Austin exec to Sam Hasting',
 'Benj Allen Executor to B Winchester',
 'Joseph Allen Executor to B Winchester',
 'William B Townsend Esq Executor to the Will of Thomas Hubbard Esq deceased',
 'To Allen Exec to Benj Winchester',
 'Robert Morris and John Simon executor to the estate of',
 'Robert Morris and John Simon executor to the estate of R Gerraty',
 'Isaac Roush and Mary Utrey executor to the estate of Jacob Carver Deceased',
 'John Edwards Executor for the Estate of Stephen Boyd',
 'David Mackey and Debt Executor',
 'George Liller exec of Leon Ampach',
 'George Kelcher and Peter Sherer Ex. To Conaard Teulps',
 'John Penny Cacker Ex. To the Estale of Ju Eighter',
 'Jacob Fisher Philip Lentz Ex.',
 'William Allison Ex. Of Mat Mease and Co.',
 'Isaac Rauch and Mary Sallie Executor of Jacob Calvert',
 'Jacob Fisher and Chris Lentz Executor of Christ V. Cross',
 'Atkinsons Executors']

In [206]:
for x in executor.loc[[39712, 39713, 40030]]['First Name 1 ']:
    print(x)

Robert Morris and John Simon executor to the estate of
Robert Morris and John Simon executor to the estate of
Robert Morris and John Simon executor to the estate of R Gerraty


In [207]:
executor_names = executor['First Name 1 ']
#handle special cases
executor_names = executor_names.replace("Atkinsons Executors", "Atkinsons")
executor_names = executor_names.replace("David Mackey and Debt Executor", "David Mackey")
executor_names = executor_names.replace("Jacob Fisher Philip Lentz Ex.", "Jacob Fisher")
executor_names = executor_names.replace("Robert Morris and John Simon executor to the estate of", "Robert Morris and John Simon executor to the estate of R Gerraty")
spec_status_names = ("David Mackey", "Atkinsons", "Jacob Fisher", "Robert Morris") 
#find string after the words to or of
post_of = executor_names.apply(lambda x: x[x.lower().find(' to '):] if x not in spec_status_names else x)
post_to = executor_names.apply(lambda x: x[x.lower().find(' of '):] if x not in spec_status_names else x)
#merge results
pre_processing_executor_names = pd.Series(x if len(x) >= len(y) else y for x, y in zip(post_of, post_to))
#remove stop words
pre_processing_executor_names = pre_processing_executor_names.apply(lambda x: x.replace("to", "").replace("To", "").replace("of", "").replace("Of"," ").strip())
pre_processing_executor_names = pre_processing_executor_names.apply(lambda x: x.replace("the", "").replace("  ", " ").strip())
pre_processing_executor_names.unique()

array(['Sam Hasting', 'B Winchester', 'Will Thomas Hubbard Esq deceased',
       'Benj Winchester', 'estate R Gerraty',
       'estate Jacob Carver Deceased', 'Stephen Boyd', 'David Mackey',
       'Leon Ampach', 'Conaard Teulps', 'Estale Ju Eighter',
       'Jacob Fisher', 'Mat Mease and Co.', 'Jacob Calvert',
       'Christ V. Cross', 'Atkinsons'], dtype=object)

In [208]:
nlp = spacy.load("en_core_web_sm")
executor_names = dict()
for entity in set(pre_processing_executor_names):
    doc = nlp(entity)
    if len(doc.ents) == 1:
        ent = doc.ents[0]
        if ent.label_ == "PERSON":
            executor_names[entity] = ent.text
        elif ent.label == "ORG":
            executor_names[entity] = ent.text
        else:
            executor_names[entity] = entity
    else:
        executor_names[entity] = entity

In [209]:
#executor_names

In [210]:
executor_names['estate Jacob Carver Deceased'] = 'Jacob Carver'
executor_names['estate R Gerraty'] = 'R Gerraty'
executor_names['Will Thomas Hubbard Esq deceased'] = 'Will Thomas Hubbard'
executor_names['Mat Mease and Co.'] = 'Mat Mease'
post_processing_executor_names = [executor_names[x] for x in pre_processing_executor_names]
post_processing_executor_index = executor['First Name 1 '].index
#post_processing_executor_names

In [211]:
executor['First Name 1 '] = post_processing_executor_names
executor['Last Name 1 '] = [x.split(" ")[-1] for x in executor['First Name 1 ']]
executor['First Name 1 '] = [" ".join(x.split(" ")[:-1]) for x in executor['First Name 1 ']]
#executor

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

<h3> Modify weird_first_names to include corrections </h3>

In [212]:
#make adjustments here
weird_first_names['First Name 1 '].loc[post_processing_executor_index] = post_processing_executor_names
weird_first_names['Last Name 1 '].loc[post_processing_executor_index] = [x.split(" ")[-1] for x in weird_first_names['First Name 1 '].loc[post_processing_executor_index]]
weird_first_names['First Name 1 '].loc[post_processing_executor_index] = [" ".join(x.split(" ")[:-1]) for x in weird_first_names['First Name 1 '].loc[post_processing_executor_index]]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the 

In [213]:
weird_first_names['notes'][[x == "Jacob" and y == "Carver" for x, y in zip(weird_first_names['First Name 1 '], weird_first_names['Last Name 1 '])]] = "Deceased, Estate"
weird_first_names['notes'][[x == "R" and y == "Gerraty" for x, y in zip(weird_first_names['First Name 1 '], weird_first_names['Last Name 1 '])]] = "Estate"
weird_first_names['notes'][[x == "Will Thomas" and y == "Hubbard" for x, y in zip(weird_first_names['First Name 1 '], weird_first_names['Last Name 1 '])]] = "Deceased"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we c

In [214]:
executor_cleaned = weird_first_names.loc[post_processing_executor_index]
filter_out_names = executor_cleaned.index.tolist()
#executor_cleaned

In [215]:
#trend below - if its two and's deal with first case below
#if its two &'s deal with second case below
#otherwise just split the names

In [216]:
two_names = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: ("&" in x or " and " in x.lower()))]
new_index = [x not in filter_out_names for x in two_names.index]
two_names = two_names.loc[new_index]
two_names.shape

(2261, 13)

<h2> Handle corrections of names with weird combinators </h2> 

In [217]:
multiple_and = two_names[two_names['First Name 1 '].apply(lambda x: x.count('and') > 1)]
#multiple_and

In [218]:
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('Sarah Charman for the use of Gilbert Hammond and Cornelius Tommand', 
                                                                      'Gilbert Hammond and Cornelius Tommand')
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('John Cavott and Jacob Aivl for Moses Dichey and George Dukey', 
                                                                      'Moses Dichey and George Dukey')
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('Jn Nixon I M Nerbitt and Alexander Forster', 
                                                                      'Jn Nixon and I M Nerbitt and Alexander Forster')
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('Samuel Ely and Michael Gellington Esq and Co',
                                                                     'Samuel Ely and Michael Gellington Esq')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

<h3> Fix by replacing in two_names, not superior weird_first_names </h3> 

In [219]:
two_names.loc[multiple_and.index] = multiple_and
multiple_and_cleaned = two_names.loc[multiple_and.index]
#multiple_and_cleaned

In [220]:
multiple_and_symbol = two_names[two_names['First Name 1 '].apply(lambda x: x.count('&') > 1)]
#executive decision made - nic jacobs is executor of peter brown
multiple_and_symbol['First Name 1 '] = multiple_and_symbol['First Name 1 '].replace('Peter Brown & Nic Jacobs Ex & …….', 
                                                                      'Peter Brown')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [221]:
two_names.loc[multiple_and_symbol.index] = multiple_and_symbol
two_names.loc[multiple_and_symbol.index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Face Value,Specie Value,notes
57469,6,1779,8,14.0,NaN,Hre Rest & Ch & St Pr Chumess,NaN,NaN,NaN,NaN,500,29.41667,NaN
57680,6,1779,9,2.0,NaN,Peter Brown,NaN,NaN,NaN,NaN,400,22.13333,NaN


In [222]:
#checking to make sure no string has both "and" and "&" as split symbol
two_names[(two_names['First Name 1 '].apply(lambda x: "&" in x and " and " in x))]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Face Value,Specie Value,notes


In [223]:
more_complicated = two_names[two_names['First Name 1 '].apply(lambda x: " in " in x or " of " in x or "to " in x or "for" in x)]
more_complicated_names = more_complicated['First Name 1 '].unique().tolist()

In [224]:
more_complicated_names

['John Gray &Tho Dawes Fws.to Sarah Green',
 'Jn Gray & Thomas Dawes Trustees to Sach Green',
 'Nathaniel Appleton and other trustees of Judah Monis Legasy',
 'John Barrett & Sons Trustees to Creditors of John Elsworth',
 'Society for Relief of Poor Masters of ships widows and children',
 'Corporation for the Relief of Poor and Distressed Presbyterian Ministers',
 'Jeremiah Halsey and Sarah Gaston for the Estate of Gaston Dec',
 'Wm Allison & Sam Caldwell Ex to the est of David Caldwell for children',
 'Jacob Brush & Mary Wroop to the est of Jacob Carver Ded',
 'Mathew Greer & Mathew Greer for the heirs of Thomas Jones Deceased',
 'Joseph Jacket and Anthany Jacket Trustees of Presbyterian Congregation N town',
 'Isaac Roush & Mary Bldney of Jacob Carver',
 'Samuel Johnston Inna Hanna and I Johnston in trust for the Hierrs of Phil Johnston',
 'Michele Shubart For Corporation of Michelle & Zion Churches',
 'Corporation for relief of poor and distressed presbyterian ministers',
 'Ed Keasb

<h2> Handle corrections of and co. </h2>

In [225]:
has_co = two_names[two_names['First Name 1 '].apply(lambda x: x[-2:] == "Co" or "co." in x.lower() or " others " in x.lower() and not betterIsNan(x))]
has_co_names = pd.Series(has_co['First Name 1 '].unique())

In [226]:
capt_sam_filter = [x == 'Capt Samuel Wilman and Co.' for x in has_co['First Name 1 ']]
has_co['First Name 1 '][capt_sam_filter] = "Samuel"
has_co['Last Name 1 '][capt_sam_filter] = "Wilman"
has_co['Title 1'][capt_sam_filter] = "Capt"

paris_bro_filter = [x == 'Paris Brothers and co.' for x in has_co['First Name 1 ']]
has_co['First Name 1 '][paris_bro_filter] = math.nan
has_co['Last Name 1 '][paris_bro_filter] = "Paris"
has_co['notes'][paris_bro_filter] = "brothers"

gs_dewint_filter = [x == 'GS.Dewint & Co' for x in has_co['First Name 1 ']]
has_co['Last Name 1 '][gs_dewint_filter] = "Dewint"
has_co['First Name 1 '][gs_dewint_filter] = "GS."

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the 

In [227]:
spec_status_names = ['Capt Samuel Wilman and Co.', 'Paris Brothers and co.', 'GS.Dewint & Co']
spec_status_first_names = ['Samuel', math.nan, 'GS.']
post_and_sym = has_co_names.apply(lambda x: x[:x.lower().find('&')] if x not in spec_status_names else x)
post_and = has_co_names.apply(lambda x: x[:x.lower().find('and')] if x not in spec_status_names else x)
#merge results
pre_processing_co_names = pd.Series(y if len(x) >= len(y) else y for x, y in zip(post_and_sym, post_and))
#remove stop words
pre_processing_co_names = pre_processing_co_names.apply(lambda x: x.replace("&", "").replace("Co.", "").replace("co."," ").strip())
pre_processing_co_names = pre_processing_co_names.apply(lambda x: x.replace(" C", "").replace(" other", "").strip())

In [228]:
replace_co = dict(pd.Series(pre_processing_co_names.tolist(), index = has_co_names))
del replace_co[spec_status_names[0]]
del replace_co[spec_status_names[1]]
del replace_co[spec_status_names[2]]

In [229]:
change_names = [x not in spec_status_first_names for x in has_co['First Name 1 ']]

In [230]:
has_co.loc[change_names, 'First Name 1 '] = has_co['First Name 1 '][change_names].apply(lambda x: replace_co[x])
has_co.loc[change_names, 'Last Name 1 '] = [x.split(" ")[-1] for x in has_co['First Name 1 '][change_names]]
has_co.loc[change_names, 'First Name 1 '] = [" ".join(x.split(" ")[:-1]) for x in has_co['First Name 1 '][change_names]]

In [231]:
two_names.loc[has_co.index, 'First Name 1 '] = has_co['First Name 1 '].tolist()
two_names.loc[has_co.index, 'Last Name 1 '] = has_co['Last Name 1 '].tolist()
#two_names.loc[has_co.index]

<h2> Handling Heirs </h2>

In [232]:
def heirPresent(x):
    try:
        return "heir" in x.lower()
    except:
        return False

In [233]:
heirs = two_names[two_names['First Name 1 '].apply(lambda x: heirPresent(x))]
heirs_names = heirs['First Name 1 '].unique()

In [234]:
heirs

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Face Value,Specie Value,notes
40272,6,1777,5,8.0,NaN,Margarett Grant herself and heirs,NaN,NaN,NaN,NaN,500,500.00000,NaN
42845,6,1778,2,25.0,NaN,Mathew Greer & Mathew Greer for the heirs of T...,NaN,NaN,NaN,NaN,500,289.26390,NaN
59863,6,1779,12,20.0,NaN,John Smith and James McDonald Guardian of the ...,NaN,NaN,NaN,NaN,400,14.28889,NaN


In [235]:
heirs.loc[40272, 'First Name 1 '] = "Margaret"
heirs.loc[40272, 'Last Name 1 '] = "Grant"
heirs.loc[40272, 'notes'] = "heirs"

heirs.loc[42845, 'First Name 1 '] = "Thomas"
heirs.loc[42845, 'Last Name 1 '] = "Jones"
heirs.loc[42845, 'notes'] = "heirs, deceased"

heirs.loc[59863, 'First Name 1 '] = "John"
heirs.loc[59863, 'Last Name 1 '] = "Gibson"
heirs.loc[59863, 'notes'] = "heirs"

In [236]:
two_names.loc[heirs.index] = heirs
two_names.loc[heirs.index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Face Value,Specie Value,notes
40272,6,1777,5,8.0,NaN,Margaret,Grant,NaN,NaN,NaN,500,500.00000,heirs
42845,6,1778,2,25.0,NaN,Thomas,Jones,NaN,NaN,NaN,500,289.26390,"heirs, deceased"
59863,6,1779,12,20.0,NaN,John,Gibson,NaN,NaN,NaN,400,14.28889,heirs


In [237]:
filter_out_names.extend(has_co.index.tolist())
filter_out_names.extend(heirs.index.tolist())
filter_out_names.extend(more_complicated.index.tolist())

<h2> Handling names separated by & and "and" </h2>

In [238]:
and_sep_filter = [x not in filter_out_names for x in two_names.index]

In [239]:
def splitNames(x):
    x = x.replace(" And ", " and ")
    if "&" in x:
        return x.split("&")
    return x.split(" and ")

In [242]:
normal_join['First Name 1 '].unique()

array(['Lee & Jones', 'Clark & Nightingale', 'Pitts & Call',
       'Bryant & Dennie', 'Hubbart & Greene', 'Hunt & Sherburne',
       'Jackson Tracy & Tracy', 'Otis & Andrews', 'Parker & Eustis',
       'Cushing & White', 'Riggs Dairs and Humphrey',
       'Henry McClellen and Henry', 'John Brinkerhoff & others',
       'Mercer and Skenk', 'John Stevens and William Burnett',
       'Forman & Hunt', 'Perrigrine & Van Ember', 'Ogden & Curti',
       'Stephen Ford and Canfield', 'Forman and Hunt',
       'Hendrick & Clears', 'Isaac & Peter Vandorn',
       'Hendrick & Jacob TenEyk', 'Prior & Joy', 'Benezet & Backman',
       'Gilbert Hammond and Cornelius Tommand', 'Simons & Henry',
       'Rotch and Jarvis', 'Moses Dichey and George Dukey',
       'Levy and Levy', 'Peter and Isaac Wikoff',
       'Peter And Isaac Wikoff', 'Milner and Haynes',
       'Michel and Zion Churches', 'Chalone and White', 'Charles & White',
       'Vance Caldwell and Vance', 'Jacob and Henry Geiger',
       'Let

In [241]:
normal_join = two_names.loc[and_sep_filter]
normal_join['First Name 1 '].apply(lambda x: splitNames(x)).apply(lambda x: len(x)).value_counts()

2    1717
3       2
1       1
Name: First Name 1 , dtype: int64

In [137]:
#added_data = pd.DataFrame(columns = normal_join.columns.tolist())

In [138]:
joinedNames = normal_join['First Name 1 '].apply(lambda x: splitNames(x))

In [143]:
joinedNames[joinedNames.apply(lambda x: len(x)>2)]

53730    [Jn Nixon, I M Nerbitt, Alexander Forster]
57469             [Hre Rest ,  Ch ,  St Pr Chumess]
Name: First Name 1 , dtype: object

In [174]:
normal_join['First Name 1 '].unique()

array([nan, 'Jackson Tracy', 'Riggs Dairs', 'Henry McClellen',
       'John Brinkerhoff', 'John', 'Perrigrine', 'Stephen Ford', 'Isaac',
       'Hendrick', 'G i l b e r t   H a m m o n', 'Moses', 'Peter',
       'Michel', 'Vance Caldwell', 'Jacob', 'Letitea', 'Lelitia', 'Ilean',
       'William', 'Robert', 'Jn', 'Sam', 'Hugh Samuel', 'John Mam',
       'Van Vrem', 'Christian', 'Rob', 'Abraham Levi', 'Hre',
       'Guilliaume Sabatier', 'John Pringle Ve', 'In Peingle Harr',
       'Will', 'Camble Cooper', 'Samuel', 'Simon Dreisbach For', 'Henry',
       'Edward', 'Massachusetts Milner', 'Harris Wood', 'Arell Cooper',
       'Polly', 'Francis Fair', 'SD George', 'Jno', 'Miss Early',
       'Messrs Newton', 'Thomas', 'Mess Early', 'Baker Blow'],
      dtype=object)

In [124]:
#length of one - then 

#length of two - then

#two one letter - two last names
#first one is one word, second is more than one word - attach last word onto 

#length of three/4 - manually handle

In [125]:
#40857 'Sarah Charman for the use of Gilbert Hammond' 1
#56384' Samuel Harris Esq Mathia Birkly' 2
#'Simon Dreisbach For Estates' 1
#61411 ['Will Robinson', 'Sons']


In [144]:
def parseNames(ind, val):
    #global added_data
    
    if len(val) == 1:
        val = val[0]
        normal_join.loc[ind, 'Last Name 1 '] = val.split(" ")[-1].strip()
        normal_join.loc[ind, 'First Name 1 '] = " ".join(val.split(" ")[:-1]).strip()
    elif len(val) == 2:
        name1 = val[0].strip().split(" ")
        name2 = val[1].strip().split(" ")
        #special cases
        if ind == 40857:
            name1 = 'Gilbert Hammond'
        if ind == 56384:
            name2 = 'Samuel Harris'
        if ind == 61411:
            normal_join.loc[ind, 'Last Name 1 '] = 'Robinson'
            normal_join.loc[ind, 'First Name 1 '] = 'Will'
            normal_join.loc[ind, 'notes'] = 'Sons'
            return
        
        #normal handling
        if len(name1) == 1 and len(name2) == 1:
            
            normal_join.loc[ind, 'notes'] = normal_join.loc[ind, 'First Name 1 ']
            
            normal_join.loc[ind, 'Last Name 1 '] = name1[0]
            normal_join.loc[ind, 'First Name 1 '] = math.nan

            normal_join.loc[ind, 'Last Name 2'] = name2[0]
            normal_join.loc[ind, 'First Name 2'] = math.nan
        elif len(name1) == 1:
            normal_join.loc[ind, 'notes'] = normal_join.loc[ind, 'First Name 1 ']
            
            normal_join.loc[ind, 'Last Name 1 '] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 1 '] = name1[0]

            normal_join.loc[ind, 'Last Name 2'] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 2'] = " ".join(name2[:-1])
        elif len(name2) == 1:
            normal_join.loc[ind, 'notes'] = normal_join.loc[ind, 'First Name 1 ']
            
            normal_join.loc[ind, 'Last Name 1 '] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 1 '] = " ".join(name1)

            normal_join.loc[ind, 'Last Name 2'] = name2[0]
            normal_join.loc[ind, 'First Name 2'] = math.nan
        else:
            normal_join.loc[ind, 'Last Name 1 '] = name1[-1].strip()
            normal_join.loc[ind, 'First Name 1 '] = " ".join(name1[:-1])
            
            normal_join.loc[ind, 'Last Name 2'] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 2'] = " ".join(name2[:-1])
    else:
        if ind == 53730:
            #53730 ['Jn Nixon, I M Nerbitt, Alexander Forster']
            normal_join.loc[ind, 'Last Name 1 '] = "Nixon"
            normal_join.loc[ind, 'First Name 1 '] = "Jn"
            
            normal_join.loc[ind, 'Last Name 2'] = "Nerbitt"
            normal_join.loc[ind, 'First Name 2'] = "I M"
                        
            normal_join.loc[ind, 'notes'] = "add Alexander Forster"
        elif ind == 57469:
            #57469 ['Hre Rest ', ' Ch ', ' St Pr Chumess']
            normal_join.loc[ind, 'Last Name 1 '] = "Rest"
            normal_join.loc[ind, 'First Name 1 '] = "Hre"
        
            normal_join.loc[ind, 'Last Name 2'] = "Chumess"
            normal_join.loc[ind, 'First Name 2'] = "St Pr"
            
            normal_join.loc[ind, 'notes'] = "add Ch"      

In [145]:
for ind, val in joinedNames.items():
    parseNames(ind, val)

In [148]:
two_names.loc[normal_join.index] = normal_join
#two_names.loc[normal_join.index]

In [149]:
weird_first_names.loc[two_names.index] = two_names

<h3> Handling treasurers </h3>

In [150]:
#society's treasurer lyme - just society 
#get organization name from words after treasurer

In [153]:
def testString(string, match):
    try:
        return match in string.lower()
    except:
        return False

In [154]:
treasurer = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: testString(x, "treas") and x not in filter_out_names)]
treasurer.shape

(70, 13)

In [155]:
filter_out_names.extend(treasurer['First Name 1 '].unique().tolist())
treasurer['First Name 1 '].unique()

array(['Ebenezer Storer Treasurer H College',
       'Job Cushing Treasurer for Shrewsbury',
       'Jos Richards Treas 2nd Precinct Roxbury',
       'Dan Thurston Treasurer Church in Franklin',
       'Simeon Howard Treasurer Convention of Ministers',
       'Eli Root Treasurer of Pittsfield', "Society's Treasurer Lyme"],
      dtype=object)

<h2> Handling everything else </h2>

In [156]:
other_cases = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: x not in two_names['First Name 1 '].tolist()
                                                                         and x not in filter_out_names)]
other_cases.shape

(1690, 13)

In [157]:
nlp = spacy.load("en_core_web_sm")
other_fixed_names = []
for entity in other_cases['First Name 1 ']:
    doc = nlp(entity)
    if len(doc.ents) == 1:
        ent = doc.ents[0]
        if ent.label_ == "PERSON" and ent.text == entity:
            other_fixed_names.append(ent.text)
        elif ent.label == "ORG" and ent.text == entity:
            other_fixed_names.append(ent.text)
        else:
            other_fixed_names.append(False)
    elif len(doc.ents) == 2:
        ent = doc.ents[0]
        ent2 = doc.ents[1]
        if ent.label_ == "Person" and ent2.label_ == "Person" and (ent.text + " " + ent2.text) == entity:
            other_fixed_names.append(ent.text + " " + ent2.text)
        else:
            other_fixed_names.append(False)
    else:
        other_fixed_names.append(False)

In [158]:
#what is william thomas exos james ecum
print(len(other_fixed_names))
set(other_fixed_names)

1690


{'Aaron Lane',
 'Abraham Abraham Haring',
 'Abraham Beachum',
 'Abraham Hite',
 'Alex White',
 'Alexa Hite',
 'Alexander Belmain',
 'Alexander St Clair',
 'Allen Otis',
 'Anderson Dugar',
 'Anderson Scott',
 'Andrew Woodrow',
 'Anna McDonald',
 'Anna Potlock',
 'Anne Blair',
 'Anne Hodges',
 'Anne Peters',
 'Anthony Baker',
 'Anthony Perryman',
 'Anthony Thompson',
 'Anthony Thornton',
 'Archd Blair',
 'Archd Ritchie',
 'Archud Bass',
 'Arthur Campbell',
 'Barrett White',
 'Benjamin Co',
 'Benjamin Day',
 'Benjamin Johnston',
 'Benjamin Robinson Jr',
 'Bernard Smith',
 'Beverley Randolph',
 'Breeson Bondoin',
 'Caleb Teakle',
 'Carter Braxton',
 'Catharine Kuykendall',
 'Catharine Scott',
 'Charles Ashton',
 'Charles Carter',
 'Charles Davison',
 'Charles Nevon',
 'Charles Thomas',
 'Charles Yates',
 'Christian Stickley',
 'Christopher Tompkins',
 'Daniel Coleman',
 'Daniel L Hylton',
 'Daniel Lannslen Bermuda',
 'Daniel Mabry',
 'Daniel Northrup',
 'Daniel Trueheart',
 'David Jameson'

<h3> Proper names that are just misplaced </h3>

In [159]:
indices = [not x if x is False else False for x in other_fixed_names]
indices_neg = [not x for x in indices]

In [160]:
swap_names = pd.Series(other_fixed_names)[indices_neg]

In [161]:
other_cases.loc[indices_neg, 'First Name 1 '] = swap_names
other_cases.loc[indices_neg, 'Last Name 1 '] = [x.split(" ")[-1] for x in swap_names]
other_cases.loc[indices_neg, 'First Name 1 '] = [" ".join(x.split(" ")[:-1]) for x in swap_names]

In [162]:
weird_first_names.loc[other_cases[indices_neg].index] = other_cases.loc[indices_neg]
weird_first_names.loc[other_cases[indices_neg].index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Face Value,Specie Value,notes
10301,2,1779,1,28.0,NaN,John Davis,Williams,NaN,NaN,NaN,500,58.791670,NaN
34954,5,1779,3,31.0,NaN,John Treat,Crane,NaN,NaN,NaN,500,45.423610,NaN
35442,5,1779,4,21.0,NaN,Samuel Wright,Hortsham,NaN,NaN,NaN,400,33.983330,NaN
35798,5,1779,5,4.0,NaN,Abraham Abraham,Haring,NaN,NaN,NaN,400,32.600000,NaN
35805,5,1779,5,4.0,NaN,Abraham Abraham,Haring,NaN,NaN,NaN,300,24.450000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80895,9,1780,2,1.0,NaN,Jesse,Taylor,NaN,NaN,NaN,300,9.020833,NaN
80906,9,1779,3,19.0,NaN,William,Bradley,NaN,NaN,NaN,1000,94.402780,NaN
80907,9,1779,3,19.0,NaN,Samuel,Oldham,NaN,NaN,NaN,1000,94.402780,NaN
80908,9,1780,1,1.0,NaN,John,Hay,NaN,NaN,NaN,400,13.616670,NaN


<h2> Handling other weird names/organizations </h2>

In [163]:
print(sum(indices))
other_cases['First Name 1 '][indices].unique()

462


array(['B', 'The Navy Board Eastern Department', 'Will Thomas',
       'Rev XXX on Behalf of Thomas Sander Esq deceased', 'Benj',
       'B Merlino De S Pry', 'William Fleet Agent Long Wharf',
       'Overseers of the Poor Town of Boston',
       'William Dunscomb by William Somersall', 'Trustees Hopkins Legacy',
       'Preston North Society', 'First Society in Stafford',
       'First Society in Saybrook', 'First Society Plainfield',
       'First Society in Lebanon', 'School Committee of Derbey',
       'New London North Parish', 'The North Society New Fairfield',
       "Fairfield Society's School", 'Plainfield First Society',
       "Litchfield First Society's Committee", 'Greenwich west society',
       'First Society in Branford', 'Heirs of Joseph Conklin',
       'Great Mill Society in Derby', 'Estate of General Montgomery',
       'Trustees of Kingston', 'Administrator of Jacob Vancys',
       'Garrett John Blauvelt', 'Cornelius John Blauvelt',
       'Middew John Phillips', '

In [164]:
#add additional row that is the same as the first when last name 2 applies
#add index indicating shared

In [169]:
(loans[[not x for x in loans['Last Name 2'].apply(betterIsNan)]][['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']].drop_duplicates()).shape

(159, 4)

In [170]:
loans[[not x for x in loans['Last Name 2'].apply(betterIsNan)]][['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']].shape

(1209, 4)

In [167]:
#checking if there are any names in first name column that are two first names
#check confirmed - also shows that there are no NA first name or last name columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(loans[[not x for x in loans['Last Name 2'].apply(betterIsNan)]][['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']].drop_duplicates())

,First Name 1,Last Name 1,First Name 2,Last Name 2
2253,Abraham,Livingston,William,Turnbull
4017,John,Gray,Thomas,Dawes
5173,Mary,Butler,Abigail,Belknap
6505,William,Molineaux,John,Molineaux
8247,Samuel,Colburn,Nathaniel,Deane
9107,Isaac,Smith,S,Hooper
11969,Asa,Waterman,John,Waterman
12876,Samuel,Salisbury,Stephen,Salisbury
13584,Sarah,White,Mary,Thayer
13838,Peter,Wikoff,Isaac,Wikoff
